## Обзор даных

### Таблица books

Ознакомимся с таблицей `books`. Но для начала подключимся к нашей базе данных, а затем с помощью языка запросов SQL выведем первые пять строк таблицы.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''

SELECT * 
FROM books 
LIMIT 5

'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица `books` содержит следующую информацию:
- `book_id` - идентификатор книги;
- `author_id` - идентификатор автора;
- `title` - название книги;
- `num_pages` - количество страниц;
- `publication_date` - дата публикации книги;
- `publisher_id` - идентификатор издателя.

### Таблица authors

Изучим данные таблицы `authors`.

In [4]:
query = '''

SELECT * 
FROM authors
LIMIT 5;

'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица `authors` содержит следующую информацию:
- `author_id` - идентификатор автора;
- `author` - имя автора.

### Таблица publishers

Посмотрим какие данные содержит таблица `publishers`.

In [5]:
query = '''

SELECT * 
FROM publishers
LIMIT 5;

'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица `publishers` содержит следующую информацию:
- `publisher_id` - идентификатор издательства;
- `publisher` - название издательства.

### Таблица ratings

Изучим данные содержит таблица `ratings`.

In [6]:
query = '''

SELECT * 
FROM ratings
LIMIT 5;

'''

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица `ratings` содержит следующую информацию:
- `rating_id` - идентификатор оценки;
- `book_id` - идентификатор книги;
- `username` - имя пользователя, оставившего оценку;
- `rating` - оценка книги.

### Таблица reviews

Посмотрим какие данные содержит таблица `reviews`.

In [7]:
query = '''

SELECT * 
FROM reviews
LIMIT 5;

'''

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица `reviews` содержит следующую информацию:
- `review_id` - идентификатор обзора;
- `book_id` - идентификатор книги;
- `username` - имя автора обзора;
- `text` - текст обзора.

## Исследовательский анализ

### Количество книг, которое вышло после 1 января 2000 года

Узнаем, какое количество книг вышло после 1 января 2000 года.

In [9]:
query = '''

SELECT COUNT(book_id) AS count_books
FROM books 
WHERE publication_date > '2000-01-01';

'''

pd.io.sql.read_sql(sql=text(query), con = con)

,count_books
0,819


**Вывод:** После 1 января 2020 года было издано 819 книг.

### Количество обзоров и средняя оценка каждой книги

Для каждой книги в нашей базе узнаем количество обзоров и среднюю оценку.

In [10]:
query = '''

WITH
re AS (SELECT b.book_id,
              COUNT(rev.review_id) AS count_reviews
       FROM books AS b
       LEFT JOIN reviews AS rev ON rev.book_id = b.book_id
       GROUP BY b.book_id),

ra AS (SELECT b.book_id,
              ROUND(AVG(rat.rating), 2) AS avg_rating
       FROM books AS b
       LEFT JOIN ratings AS rat ON rat.book_id = b.book_id
       GROUP BY b.book_id)

SELECT re.book_id,
       re.count_reviews,
       ra.avg_rating
FROM re JOIN ra ON re.book_id = ra.book_id;

''' 

pd.io.sql.read_sql(text(query), con = con)

,book_id,count_reviews,avg_rating
0,1,2,3.67
1,2,1,2.50
2,3,3,4.67
3,4,2,4.50
4,5,4,4.00
...,...,...,...
995,996,3,3.67
996,997,3,3.40
997,998,4,3.20
998,999,2,4.50


**Вывод:** В нашей базе 1000 книг, на каждую книгу приходится небольшое количество обзоров - 1-4 обзора на книгу, каждая книга оцениватся по 5 бальной шкале.

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

Узнаем, какое издательство выпустило наибольшее число книг толще 50 страниц.

In [11]:
query = '''

SELECT p.publisher,
       COUNT(b.book_id) AS count_books
FROM books AS b
JOIN publishers AS p ON p.publisher_id = b.publisher_id
WHERE num_pages > 50
GROUP BY p.publisher
ORDER BY count_books DESC
LIMIT 1;

''' 

pd.io.sql.read_sql(text(query), con = con)

,publisher,count_books
0,Penguin Books,42


**Вывод:** Издательство `Penguin Books` выпустило самое большое количество книг - 42 книги. Именно издательство `Penguin Books` выпускало серию книг о Гарри Поттере.

### Автор с самой высокой средней оценкой книг

Узнаем автора с самой высокой средней оценкой книг, с учётом только тех книг у которых количество оценок - 50 и более.

In [12]:
query = '''

WITH
r AS (SELECT book_id,
             AVG(rating) AS avg_rating
      FROM ratings
      GROUP BY book_id
      HAVING COUNT(rating) >= 50),

aut AS (SELECT b.book_id,
               a.author
        FROM books AS b
        JOIN authors AS a ON a.author_id = b.author_id) 

SELECT aut.author,
       ROUND(AVG(avg_rating), 6) AS avg_rating_books
FROM r JOIN aut ON r.book_id = aut.book_id
GROUP BY aut.author
ORDER BY avg_rating_books DESC
LIMIT 1;

''' 

pd.io.sql.read_sql(text(query), con = con)

,author,avg_rating_books
0,J.K. Rowling/Mary GrandPré,4.283844


**Вывод:** У `J.K. Rowling` и `Mary GrandPré` самый высокий средний рейтинг оценки книг - 4.28. Это и неудивительно, ведь `J.K. Rowling` и `Mary GrandPré` являются автором и иллюстратором серии книг о Гарри Поттере.

### Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

Найдём среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [13]:
query = '''

WITH
cr AS (SELECT COUNT(review_id) AS count_reviews
       FROM reviews
       GROUP BY username
       HAVING username IN (SELECT username
                           FROM ratings
                           GROUP BY username
                           HAVING COUNT(rating) > 48))


SELECT AVG(cr.count_reviews) AS avg_count_reviews
FROM cr;

''' 

pd.io.sql.read_sql(text(query), con = con)

,avg_count_reviews
0,24.0


**Вывод:** Пользователи, которые поставили больше 48 оценок, в среднем оставляли по 24 обзору.

## Общий вывод

**На этапе исследовательского анализа узнали, что:**

*1) После 1 января 2020 года было издано 819 книг.*

*2) В нашей базе 1000 книг, на каждую книгу приходится небольшое количество обзоров - 1-4 обзора на книгу, каждая книга оцениватся по 5 бальной шкале.*

*3) Издательство `Penguin Books` выпустило самое большое количество книг - 42 книги.*

*4) `J.K. Rowling` и `Mary GrandPré` обладают самым высоким средним рейтингом книг - 4.28.*

*5) Пользователи, которые поставили больше 48 оценок, в среднем оставляли по 24 обзору.*